In [1]:
pip install networkx


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install haversine


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from script import df_mobpro_brut, arr_marseille, arr_paris, arr_lyon, contours_comm, transport_dict, contours_comm, flux, plot_flux_gradient, plot_flux_gradient_zoom, coord_villes
from script import df_dossier_complet_brut, mairies, df_epci_contours, df_flux_jaune_destination_m,df_flux_rouge_destination_m,df_flux_vert_destination_m, df_flux_rouge_depart_m,df_flux_jaune_depart_m,df_flux_vert_depart_m
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.colors as colors
import networkx as nx
from scipy.sparse import coo_matrix
from shapely.geometry import Point
from shapely.ops import unary_union
from haversine import haversine

/home/onyxia/work/statapp/script.py:19: DtypeWarning: Columns (1,1687,1688) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dossier_complet_brut = pd.read_csv(file)
/home/onyxia/work/statapp/script.py:24: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta_dossier_complet_brut = pd.read_csv(file)
/home/onyxia/work/statapp/script.py:34: DtypeWarning: Columns (1,2,3,4,14,15,16,18,22,24,27,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mobpro_brut = pd.read_csv(file)
/home/onyxia/work/statapp/script.py:130: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  contours_comm["centroid"] = contours_comm.geometry.centroid
/home/onyxia/work/statapp/script.py:139: UserWarning: Geometry is in a geographic CRS. Results from 'centroid'

antully: (46.89232199838103, 4.402852764409862)
le tremblay-omonville: (49.12081156432765, 0.9143802127565656)
vaux-lès-mouzon: (49.60389494206806, 5.136483053287144)
anchenoncourt-et-chazel: (47.8616809003776, 6.11534440434135)
culoz-béon: (45.85633732729717, 5.772905566192263)
domèvre-sur-avière: (48.22223470040439, 6.388423941842605)
bernoy-le-château: (49.32120314158208, 3.3070156677330558)
videcosville: (49.566672646936, -1.3714795795608266)
saint-agrève: (45.00542393638767, 4.413096930429594)
le bourg-d'oisans: (45.0340588974907, 6.032836629436181)


In [5]:
mairies[['lat', 'lon']] = mairies['Coordonnées'].str.split(',', expand=True).astype(float)
mairies_unique = mairies.drop_duplicates(subset='Code INSEE')

# Calculer le trajet interne
def approx_internal_distance(geom):
    bounds = geom.bounds  # (minx, miny, maxx, maxy)
    point_min = (bounds[1], bounds[0])  # (lat_min, lon_min)
    point_max = (bounds[3], bounds[2])  # (lat_max, lon_max)
    return haversine(point_min, point_max)

contours_comm['distance_intra'] = contours_comm['geometry'].apply(approx_internal_distance)/2 # On divise par 2 la diag pour les intra
distance_intra_dict = contours_comm.set_index('INSEE_COM')['distance_intra'].to_dict() # Dictionnaire des distances intra

# Calculer les trajets entre villes
mairies_unique = mairies.drop_duplicates(subset='Code INSEE') # Garder une seule mairie pour chaque ville
coords_mairies = mairies_unique.set_index('Code INSEE')[['lat', 'lon']].to_dict('index') 

# Fonction pour calculer la distance
def calculer_distance(code_depart, code_arrivee):
    if code_depart == code_arrivee:
        # Trajet intra-communal
        return distance_intra_dict.get(code_depart, 3.0)  # Fallback 3km si pas trouvé
    else:
        # Trajet intercommunal
        coords_depart = coords_mairies.get(code_depart)
        coords_arrivee = coords_mairies.get(code_arrivee)
        
        if coords_depart is None or coords_arrivee is None:
            return None  # Impossible de calculer (missing coords)
        
        return haversine((coords_depart['lat'], coords_depart['lon']),
                         (coords_arrivee['lat'], coords_arrivee['lon']))

df_mobpro_brut['distance_km'] = df_mobpro_brut.apply(
    lambda row: calculer_distance(row['COMMUNE'], row['DCLT']),
    axis=1)


In [7]:
df_mobpro_brut.head(20)

,Unnamed: 0,COMMUNE,ARM,DCFLT,DCLT,AGEREVQ,CS1,DEROU,DIPL,EMPL,...,STAT,STOCD,TP,TRANS,TYPL,TYPMR,VOIT,TRANS_LABEL,CS_LABEL,distance_km
0,0,01001,ZZZZZ,99999,01001,40,2,Z,16,21,...,21,21,1,1,1,41,2,Pas de transport,"Artisans, commerçants, chef de société",4.163392
1,1,01001,ZZZZZ,99999,01001,45,5,Z,14,16,...,10,22,2,5,2,12,1,"Voiture, camion, fourgonnette",Employés,4.163392
2,2,01001,ZZZZZ,99999,01001,20,2,Z,14,21,...,21,10,1,1,1,41,3,Pas de transport,"Artisans, commerçants, chef de société",4.163392
3,3,01001,ZZZZZ,99999,01001,55,2,Z,3,21,...,21,10,1,1,1,41,1,Pas de transport,"Artisans, commerçants, chef de société",4.163392
4,4,01001,ZZZZZ,99999,01001,55,2,Z,13,22,...,22,10,1,5,1,41,3,"Voiture, camion, fourgonnette","Artisans, commerçants, chef de société",4.163392
5,5,01001,ZZZZZ,99999,01001,55,1,Z,15,21,...,21,10,1,5,1,41,2,"Voiture, camion, fourgonnette",Agriculteurs,4.163392
6,6,01001,ZZZZZ,99999,01001,30,5,Z,14,16,...,10,10,1,5,1,41,2,"Voiture, camion, fourgonnette",Employés,4.163392
7,7,01001,ZZZZZ,99999,01001,30,6,Z,15,16,...,10,10,1,5,1,41,2,"Voiture, camion, fourgonnette",Ouvriers,4.163392
8,8,01001,ZZZZZ,99999,01001,25,5,Z,14,16,...,10,10,1,1,1,41,2,Pas de transport,Employés,4.163392
9,9,01001,ZZZZZ,99999,01001,35,4,Z,17,16,...,10,10,1,5,1,41,2,"Voiture, camion, fourgonnette",Professions intermédiaires,4.163392


In [8]:
def est_mono_departement(liste_departements):
    '''
    On vérifie si les EPCI regroupent forcément des villes qui appartiennent au même département (réponse = non pas systématiquement)
    '''
    return len(set(liste_departements)) == 1
    
    
# Appliquer la fonction à chaque ligne et créer une nouvelle colonne
df_epci_contours['MONO_DEP'] = df_epci_contours['INSEE_DEP'].apply(est_mono_departement)